In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
#from tensorflow.keras.optimizers import RMSprop


In [2]:
dataset=pd.read_csv('SolarPrediction_aligned_Sun.csv',engine='python',nrows=576*15)
dataset=dataset.drop("Data",axis=1)
dataset=dataset.drop("Time",axis=1)
# dataset=dataset.drop("TimeSunRise",axis=1)
# dataset=dataset.drop("TimeSunSet",axis=1)
target=dataset["Radiation"]
dataset.head(5)

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,8,2.58,51,30.43,103,77.27,11.25,22020,67080
1,310,2.83,51,30.43,103,153.44,9.00,22020,67080
2,1206,2.16,51,30.43,103,142.04,7.87,22020,67080
3,1505,2.21,51,30.43,103,144.12,18.00,22020,67080
4,1809,2.25,51,30.43,103,67.42,11.25,22020,67080


In [3]:
dataset=dataset.values

dataset=dataset.astype('float32')

scalar1=MinMaxScaler(feature_range=(0,1))
scalar2=MinMaxScaler(feature_range=(0,1))
scalar_dim=dataset[:,1]
dataset=scalar1.fit_transform(dataset)

scalar_dim=scalar_dim.reshape(len(dataset),1)
print(scalar_dim)
scalar_dim=scalar2.fit_transform(scalar_dim)
print(scalar_dim)

[[  2.58]
 [  2.83]
 [  2.16]
 ...
 [344.9 ]
 [365.66]
 [355.57]]
[[0.00089369]
 [0.00104993]
 [0.00063121]
 ...
 [0.21482898]
 [0.2278031 ]
 [0.22149728]]


In [4]:
train_size=int(len(dataset)*0.67)
test_size=len(dataset)- train_size
train,test=dataset[0:train_size,:],dataset[train_size:len(dataset),:]



In [5]:
def create_dataset(dataset,look_back):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        #a=dataset第0行i~i+look_back個
        #參考資料數9個
        a=dataset[i:(i+look_back),0:9]
        dataX.append(a)
        dataY.append(dataset[i+look_back,1])
    return np.array(dataX),np.array(dataY)

In [6]:
#根據前面幾個資料來看下一步 彙整成look_back行的資料
look_back=5
trainX,trainY=create_dataset(train,look_back)
testX,testY=create_dataset(test,look_back)

In [7]:
model=Sequential()
model.add(Dense(5,activation='relu',input_shape=(look_back,9)))
model.add(Dense(1,activation='linear'))

model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['mean_squared_error'])


激活函數使用linear 即可做回歸預測
用以分類則最後一層使用分類個數+sigmoid之類的分類器

In [8]:
# model = MLPRegressor(hidden_layer_sizes=(100, ), max_iter=1000, random_state=42)
# model.fit(trainX, trainY)
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)


Epoch 1/10
5782/5782 - 8s - loss: 0.0121 - mean_squared_error: 0.0121 - 8s/epoch - 1ms/step
Epoch 2/10
5782/5782 - 7s - loss: 0.0070 - mean_squared_error: 0.0070 - 7s/epoch - 1ms/step
Epoch 3/10
5782/5782 - 7s - loss: 0.0065 - mean_squared_error: 0.0065 - 7s/epoch - 1ms/step
Epoch 4/10
5782/5782 - 7s - loss: 0.0063 - mean_squared_error: 0.0063 - 7s/epoch - 1ms/step
Epoch 5/10
5782/5782 - 7s - loss: 0.0062 - mean_squared_error: 0.0062 - 7s/epoch - 1ms/step
Epoch 6/10
5782/5782 - 8s - loss: 0.0062 - mean_squared_error: 0.0062 - 8s/epoch - 1ms/step
Epoch 7/10
5782/5782 - 7s - loss: 0.0062 - mean_squared_error: 0.0062 - 7s/epoch - 1ms/step
Epoch 8/10
5782/5782 - 8s - loss: 0.0061 - mean_squared_error: 0.0061 - 8s/epoch - 1ms/step
Epoch 9/10
5782/5782 - 8s - loss: 0.0061 - mean_squared_error: 0.0061 - 8s/epoch - 1ms/step
Epoch 10/10
5782/5782 - 8s - loss: 0.0061 - mean_squared_error: 0.0061 - 8s/epoch - 1ms/step


In [9]:
#model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
#model.fit(trainX, trainY, epochs=100, batch_size=128, validation_data=(testX, testY))


In [11]:
testPre = model.predict(testX)
trainPre=model.predict(trainX)

181/181 [==============================] - 0s 1ms/step


In [12]:
print(trainPre[0,:,:])
print(trainY[0])
print(trainX[0])

[[0.03430951]
 [0.03197417]
 [0.03073387]
 [0.03561379]
 [0.03327753]]
0.00062495715
[[5.8056499e-05 8.9368853e-04 3.3333325e-01 4.7367859e-01 1.0000001e+00
  2.1446240e-01 5.5555558e-01 0.0000000e+00 1.0000000e+00]
 [3.5646691e-03 1.0499278e-03 3.3333325e-01 4.7367859e-01 1.0000001e+00
  4.2614573e-01 4.4444445e-01 0.0000000e+00 1.0000000e+00]
 [1.3968393e-02 6.3120667e-04 3.3333325e-01 4.7367859e-01 1.0000001e+00
  3.9446408e-01 3.8864198e-01 0.0000000e+00 1.0000000e+00]
 [1.7440172e-02 6.6245452e-04 3.3333325e-01 4.7367859e-01 1.0000001e+00
  4.0024456e-01 8.8888890e-01 0.0000000e+00 1.0000000e+00]
 [2.0970007e-02 6.8745273e-04 3.3333325e-01 4.7367859e-01 1.0000001e+00
  1.8708836e-01 5.5555558e-01 0.0000000e+00 1.0000000e+00]]


In [13]:
trainPre=scalar2.inverse_transform(trainPre)
trainY_ord=scalar2.inverse_transform([trainY])
testPre=scalar2.inverse_transform(testPre)
testY_ord=scalar2.inverse_transform([testY])

ValueError: Found array with dim 3. None expected <= 2.

In [ ]:
#print(X_test)
print(testPre)
print(testY)

[[[0.00380274]
  [0.00380274]
  [0.00380274]
  [0.00380274]
  [0.00380274]]

 [[0.00380274]
  [0.00380274]
  [0.00380274]
  [0.00380274]
  [0.00380274]]

 [[0.00380274]
  [0.00380274]
  [0.00380274]
  [0.00380274]
  [0.00380274]]

 ...

 [[0.0645123 ]
  [0.08761235]
  [0.13217151]
  [0.13099615]
  [0.09615608]]

 [[0.08761235]
  [0.13217151]
  [0.13099615]
  [0.09615608]
  [0.1201393 ]]

 [[0.13217151]
  [0.13099615]
  [0.09615608]
  [0.1201393 ]
  [0.19303675]]]
[4.3747015e-05 5.6246121e-05 5.6246121e-05 ... 1.3781552e-01 2.1482898e-01
 2.2780310e-01]


In [ ]:
trainScore=math.sqrt(mean_squared_error(trainY,trainPre))
print('Train Score:%.2f RMSE'%(trainScore))
testScore=math.sqrt(mean_squared_error(testY,testPre))
print('Test Score:%.2f RMSE'%(testScore))
MAE=mean_absolute_error(testY,testPre)
print('Test Score:%.2f MAE'%(MAE))

In [ ]:
plt.plot(testY,label='Ground Truth')
plt.plot(testPre,label='Prediction')
#plt.plot(testPredictPlot[0:200],label='Prediction(testing)')
plt.legend()
plt.show()